### TODO: Discussion of Decision Tree and Problem

<a href=#MemoryBound>Memory Bound</a>

L1: L1 can do partial loads, cannot assume full cacheline. Look to see how many loads/cycle are being completed: `L1_Hit_Rate * LD_Instructions / Application_Cycles = L1_Loads_per_Cycle` vs. # Load Ports Available

<a name='MemoryBound'/></a>

## Memory Bound 

#### Memory Bandwidth Bound?

1. <a href=#Roofline>Measure system with Empirical Roofline Toolkit</a>
2. <a href=#HPCToolkit>Measure application with HPCToolkit</a>
3. <a href=#HPCViewer>Collect the data with HPCViewer</a>
4. <a href=#Analyze>Analyze the data</a>



<a name='Roofline'/></a>
### 1. Measure the System with Empirical Roofline ToolKit
```
git clone https://bitbucket.org/berkeleylab/cs-roofline-toolkit.git
cd cs-roofline-toolkit/Empirical_Roofline_Tool-1.1.0
```
// TODO: Edit/Rename existing config file in Config directory // Explain some values or point to ERT Doc?  
```
./ert /Path/to/your/config.file
```
View the .ps file in the results directory
![](assets/roofline.png)
Extract the Data

In [5]:
# Data From Roofline
Roofline_Bandwidth = {}
Roofline_Bandwidth['L1'] = 2399.9 # GB/s
Roofline_Bandwidth['L2']  = 784.0 # GB/s
Roofline_Bandwidth['L3']  = 564.8 # GB/s
Roofline_Bandwidth['DRAM']  = 60.4 # GB/s

#### Collect Additional System Data

In [6]:
# Collect information from other sources
BYTES_per_CACHELINE = 64 # Bytes per Cacheline # From Documentation
CPU_MHz = 2301.0 # MHz # From lscpu or /proc/cpuinfo

<a name='HPCToolkit'/></a>
### 2. Measure the Application with HPCToolkit
#### Build Application with Debug Symbols and Optimization Options
```
clang -g -O3 ...
```
#### View Available Events:  
`hpcrun -L <.exe>`
![](assets/hpcrunList.png)
#### Choose Desired Events and Measure
```
hpcrun -e <event1> -e <event2> ... -e <eventN> \
    -o MeasurementDirectory <.exe> [application options]
```
Repeat `hpcrun` as needed to collect all desired measurements. Notes:
1. When collecting same measurement into same measurement directory it sums them into one value.
2. Measure Total Cycles by itself, counter can be effected by others
    TODO: Figure out why exactly
3. Total Cycles is sum of all threads
4. Some events may conflict without proper error message

#### Build Binary Structure File
```
hpcstruct <.exe>
```
#### Build HPCToolkit Database
```
hpcprof -S <.exe>.hpcstruct -I /Path/to/Src/'+' -o OutputDirectory MeasurementDirectory
```
*Note: '+' Recursively Searches that Directory*

<a name='HPCViewer'></a>
### 3. Collect the Data with HPCToolkit //  TODO: point to their doc?
`hpcviewer /Path/to/OutputDirectory`  
![](Hotpath or whole thing?)
Where is it spending its time.  
Grab Data from Important Part.  **Can't automate easily**


<a name='Analyze'/></a>
### 4. Analyze the Data

In [7]:
# Measurements from HPCToolkit

# TODO: Discuss the various ways to count this?
BW_Used = {}
BW_Used['L2'] = L1_misses = 1.52e+10 # Cachelines
BW_Used['L3'] = L2_misses = 1.06e+10 # Cachelines
BW_Used['DRAM'] = L3_misses = 1.04e+09 # Cachelines

Total_Cycles = 1.42e+12 # cycles

Threads = 1

In [8]:
# Conversion Constants
BYTES_per_GB = 1073741824
CYCLES_per_SEC_per_MHz = 1000000

In [9]:
# Do Some Conversions

# Measured Cycles to Seconds
CYCLES_per_SEC = CPU_MHz * CYCLES_per_SEC_per_MHz
Application_Run_Cycles = Total_Cycles / Threads # Total Cycles counted on each thread
Application_Seconds = Application_Run_Cycles / CYCLES_per_SEC

# Convert HPCToolkit Measurement to GB/sec
# Cachelines to Bytes to GBs / Cycles to sec
for CacheName, Cachelines in BW_Used.items():
    Bytes_Used = Cachelines * BYTES_per_CACHELINE # Bytes
    GB_Used = Bytes_Used / BYTES_per_GB # GBs
    Application_BW = GB_Used / Application_Seconds # GB/sec

    # Compare with roofline results
    Utilization = Application_BW / Roofline_Bandwidth[CacheName]
    print('{} Bandwidth Used: {:.2f}%'.format(CacheName, Utilization * 100))

L2 Bandwidth Used: 0.19%
L3 Bandwidth Used: 0.18%
DRAM Bandwidth Used: 0.17%


If bandwidth utilization is low, but application is bound on memory then most likely bound by memory latency.